In [1]:
import json

import matplotlib.pyplot as plt
import numpy as np

from histalign.backend.ccf.paths import get_atlas_path
from histalign.backend.io import load_alignment_settings, load_image, load_volume
from histalign.backend.models import (
    AlignmentSettings,
    HistologySettings,
    VolumeSettings,
)
from histalign.backend.registration import Registrator
from histalign.backend.workspace import VolumeSlicer

In [2]:
def imshow(image: np.ndarray, title: str = "") -> None:
    figure, axes = plt.subplots()

    figure.suptitle(title)

    axes.imshow(image)
    axes.axis(False)

    plt.tight_layout()
    plt.show()

In [3]:
alignment_settings = load_alignment_settings(
    "/home/ediun/git/histalign/tests/registration/resources/complete_alignment_settings.json"
)

image = np.load(
    "/home/ediun/git/histalign/tests/registration/resources/A2_mcherry555_mecp488_dapi_image0000_channel2_maximum_uint8.npz"
)["array"]
volume = load_volume(alignment_settings.volume_path)

registrator = Registrator(True, True)

In [4]:
for parameter in [
    "scale",
    "shear",
    "rotation",
    "translation",
    "offset",
    "pitch",
    "yaw",
    "complete",
]:
    current_settings = load_alignment_settings(
        f"/home/ediun/git/histalign/projects/project_registration_test/12c22ee785/"
        f"{parameter}_alignment_settings.json"
    )

    forwarded_image = registrator.get_forwarded_image(image, current_settings)
    volume_image = VolumeSlicer(volume=volume).slice(current_settings.volume_settings)

    forward_composite_image = np.where(
        forwarded_image > 10, forwarded_image * 3, volume_image
    )
    np.savez_compressed(
        f"/home/ediun/git/histalign/tests/registration/resources/"
        f"{parameter}_expected_output1.npz",
        array=forward_composite_image,
    )

    reversed_image = registrator.get_reversed_image(current_settings, "atlas", image)

    reverse_composite_image = np.where(image > 10, reversed_image, 0)

    np.savez_compressed(
        f"/home/ediun/git/histalign/tests/registration/resources/"
        f"{parameter}_expected_output2.npz",
        array=np.where(reversed_image, reversed_image, image * 3),
    )

    with open(
        f"/home/ediun/git/histalign/tests/registration/resources/"
        f"{parameter}_alignment_settings.json",
        "w",
    ) as handle:
        dump = current_settings.model_dump()

        dump["volume_path"] = ""
        dump["histology_path"] = (
            "tests/registration/resources/"
            "A2_mcherry555_mecp488_dapi_image0000_channel2_maximum_uint8.npz"
        )

        json.dump(dump, handle)